# Other analyses

This interactive notebook allows for the generation of Supplementary Table 2 (the mean deviation in gaze position along the X and Y axis across runs of retinotopic mapping stimuli presentation and individuals from each cluster) and the aggregation of demographic information and freesurfer output to a .csv for LME fitting.

## General settings

In [1]:
import numpy as np
import pandas as pd
import random
from mne.io.eyelink import read_raw_eyelink
from ipywidgets import interact, Dropdown

In [2]:
# All individuals
with open('./../list_subj.txt') as fp:
    subjects = fp.read().split("\n")
list_of_subs = np.array(subjects[0:len(subjects) - 1])

# Cluster assignment
cluster_assignment = np.load('./../output/clusters_individualIndeces_PA.npz')['list']

clusters = {}
for i in np.unique(cluster_assignment):
    clusters['cluster_' + str(i + 1)] = list(list_of_subs[cluster_assignment == i])

## Eye-tracking: gaze position

This visualization function assumes .asc files are available in *./../data/eyetracking/*. Note we provide a bash script for downloading data using AWS command line interface. To do so, you must ensure [awscli](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#cliv2-linux-install) is installed. Then you can run 'aws configure' on a Terminal window and add your HCP credentials.

In [61]:
cluster_index = Dropdown(options = clusters.keys())
subject_index= Dropdown()

@interact(cluster = cluster_index, index = subject_index)
def plot(cluster, index):
    subject_index.options = clusters[cluster]
    if index == None:
        return 'Select a cluster and a subject'
    else:
        raw = read_raw_eyelink('./../data/eyetracking/' + str(index) + '_7T_RETBAR1_eyetrack.asc',
                       create_annotations=None)
        custom_scalings = dict(eyegaze=1e3)
        return raw.pick_types(eyetrack=True).plot(duration=400,scalings=custom_scalings)

interactive(children=(Dropdown(description='cluster', options=('cluster_1', 'cluster_2', 'cluster_3', 'cluster…

The eyetrackingParticipantAveragesXY.csv file contains the average deviation from the fixation cross for each subject for both X and Y axes. The average deviation is calculated by taking the mean deviation along each direction 
over time and retinotopic mapping run. The script for generating this file is located in *./../matlab/*, and is called 'eyetrackingParticipantAveragesXY.m'. This file is also available on OSF (https://osf.io/tdkuj/)

In [4]:


def averageDev(cluster):
    # load csv file
    df = pd.read_csv('./../output/eyetrackingParticipantAveragesXY.csv')
    x_dev = []
    y_dev = []
    for sub in cluster:
        for j in range(len(df['participantID'])):
            if str(sub) == str(df['participantID'][j]):
                x_dev.append(df['deviationX'][j])
                y_dev.append(df['deviationY'][j])
                pass
            else:
                pass
    return np.mean(x_dev), np.std(x_dev), np.mean(y_dev), np.std(y_dev), len(x_dev)


In [5]:
data = {'Cluster':[],
        'Mean X average deviation':[], 
        'Std X average deviation':[], 
        'Mean Y average deviation':[],
        'Std Y average deviation':[],
        'Number of participants':[]}

for i in range(6):
    X, std_X, Y, std_Y, sample = averageDev(clusters['cluster_' + str(i + 1) ])
    data['Cluster'].append('cluster ' + str(i + 1))
    data['Mean X average deviation'].append(X)
    data['Std X average deviation'].append(std_X)
    data['Mean Y average deviation'].append(Y)
    data['Std Y average deviation'].append(std_Y)
    data['Number of participants'].append(sample)

df = pd.DataFrame(data)
# df.to_excel('./output/averageDeviation.xlsx')
df

,Cluster,Mean X average deviation,Std X average deviation,Mean Y average deviation,Std Y average deviation,Number of participants
0,cluster 1,79.995649,59.260624,101.195545,92.827950,27
1,cluster 2,43.527691,29.526326,66.071926,79.101049,19
2,cluster 3,58.655426,49.396940,71.985604,73.123294,22
3,cluster 4,65.498358,98.806874,70.606668,93.594094,47
4,cluster 5,63.495065,49.506019,59.790594,78.149133,21
5,cluster 6,53.148597,31.888423,81.161360,92.243533,11


## Other covariates

Here we concatenate age, gender, and hemispheric grey matter volume to a .csv file for LME model fitting.

In [5]:
# Demographic data and FreeSurfer output are available on ConnectomeDB (https://db.humanconnectome.org/app/template/Login.vm) 
# under the following project: HCP 1200 Subjects Data Release > Subjects with 7T MR Session data.

demographic_data = pd.read_csv('./../RESTRICTED_HCPdata.csv')
fs_data = pd.read_csv('./../unrestricted_freesurfer_HCPdata.csv')
unrestricted_demographic_data = pd.read_csv('./../unrestricted_demographics_HCPdata.csv')

In [21]:
# Demographic info
age = demographic_data['Age_in_Yrs']
subs = demographic_data['Subject'].array
gender = unrestricted_demographic_data['Gender']

# Freesurfer data
left_GM_vol = fs_data['FS_LCort_GM_Vol']
right_GM_vol = fs_data['FS_RCort_GM_Vol']

# To check if all subjects are in the same order
new_list_subs = np.array(subs) 

# Remove subjects that are not in the HCP 7T retinotopy dataset
list_of_subs = list_of_subs.astype(int)
for sub in subs:
    if sub not in list_of_subs:
        age.drop([(subs==sub).argmax()], axis = 0, inplace = True)
        left_GM_vol.drop([(subs==sub).argmax()], axis=0, inplace = True)
        right_GM_vol.drop([(subs==sub).argmax()], axis=0, inplace = True)
        gender.drop([(subs==sub).argmax()], axis=0, inplace = True)
        new_list_subs = np.delete(new_list_subs, np.where(new_list_subs == sub))


In [47]:
# Check subject list
retinotopic_map = 'polarAngle' # 'polarAngle', 'eccentricity'
df_main = pd.read_csv('./../output/lme/longFormat_' + retinotopic_map + '_plusCovariates_MSMall_all.csv')
np.sum(np.array(df_main['HCP_ID'][0:181]).astype(int)==list(new_list_subs.astype(int)))

181

In [52]:
age = pd.concat([age]*12,axis=0).reset_index() # 12 = n_visual_areas * n_hemispheres * n_portions
GM_vol = pd.concat([left_GM_vol, right_GM_vol]*6,axis=0).reset_index() # 6 = n_visual_areas * n_portions
gender = pd.concat([gender]*12,axis=0).reset_index() # 12 = n_visual_areas * n_hemispheres * n_portions

retinotopic_map = 'polarAngle' # 'polarAngle', 'eccentricity'
df_main = pd.read_csv('./../output/lme/longFormat_' + retinotopic_map + '_plusCovariates_MSMall_all.csv').reset_index()

new_df = pd.concat([df_main, age, GM_vol, gender], axis=1)
new_df.to_csv('./../output/lme/longFormat_' + retinotopic_map + '_plusCovariates_plusAgeGenderGMvol_MSMall_all.csv')
new_df.to_excel('./../output/lme/longFormat_' + retinotopic_map + '_plusCovariates_plusAgeGenderGMvol_MSMall_all.xlsx')

Final LME model fit output is available on OSF as *longFormat_polarAngle_plusCovariates_plusAgeGenderGMvol_MSMall_all.pdf
*. Note we cannot provide sensitive information.